<a href="https://colab.research.google.com/github/kibbm/ML-Tensorflow/blob/master/%EC%99%80%EC%9D%B8%ED%92%88%EC%A7%88%EB%93%B1%EA%B8%89%EC%98%88%EC%B8%A1_%EA%B8%B0%EC%88%A0%ED%86%B5%EA%B3%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ch7.1 [기술통계 분석 + 그래프]
#Wine 품질등급예측*
---
*   목표: 와인 속성을 분석하여 품질 등급을 예측
*   핵심개념: 기술 통계, 회귀분석, t-검정, 히스토그램
*   데이터 수집: 레드와인/화이트 와인 dataset
*   데이터 준비: 수집한 데이터 파일 병합
*   데이터 탐색: 
      1.   정보확인: info()
      2.   기술 통계 확인: describe(), unique(), value_counts()
*   데이터 모델링: 
      1.   데이터를 두 그룹으로 비교 분석
        *   그룹별 기술 통계 분석: describe()
        *   t-검정: scipy 패키지의 ttest_ind()
        *   회귀분석: statsmodels.formula.api패키지의 ols()
      2.   품질 등급 예측
        *   샘플을 독립변수(x)로 지정-> 회귀분석모델 적용-> 종속변수(y)인 품질(quality)예측
*   데이터 시각화
      1.   히스토그램을 이용한 시각화
      2.   부분 회귀 플롯을 이용한 시각화

#1. 데이터 준비

In [4]:
#1.1 다운로드한 CSV파일 정리하기 : 엑셀에서 열 구분자를 세미콜론으로 인식시키기.
import pandas as pd

red_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/highSchool_bigData/data/winequality-red.csv', sep=';', header=0, engine='python')
white_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/highSchool_bigData/data/winequality-white.csv', sep=';', header=0, engine='python')

red_df.to_csv('/content/drive/MyDrive/Colab Notebooks/highSchool_bigData/data/winequality-red2.csv', index=False)
white_df.to_csv('/content/drive/MyDrive/Colab Notebooks/highSchool_bigData/data/winequality-white2.csv', index=False)

#2. 데이터 병합

In [5]:
#2.1 red wine 데이터 
red_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
red_df.insert(0, column='type', value='red')
red_df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,red,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,red,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,red,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,red,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,red,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [7]:
red_df.shape

(1599, 13)

In [8]:
#2.2 white wine 데이터 
white_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [9]:
white_df.insert(0, column='type', value='white')
white_df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [10]:
white_df.shape

(4898, 13)

In [11]:
#2.3 합치기
wine=pd.concat([red_df, white_df])
wine.shape

(6497, 13)

In [12]:
wine.to_csv('/content/drive/MyDrive/Colab Notebooks/highSchool_bigData/data/wine.csv', index=False)

#3. 데이터 탐색

In [13]:
#3.1 기본 정보 확인: info()
print(wine.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6497 entries, 0 to 4897
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6497 non-null   float64
 2   volatile acidity      6497 non-null   float64
 3   citric acid           6497 non-null   float64
 4   residual sugar        6497 non-null   float64
 5   chlorides             6497 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6497 non-null   float64
 10  sulphates             6497 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 710.6+ KB
None


In [14]:
#3.2 함수를 이용해 기술 통계 확인: describe(), unique(), value_counts()
wine.columns = wine.columns.str.replace('', '_')

wine.head()

,_t_y_p_e_,_f_i_x_e_d_ _a_c_i_d_i_t_y_,_v_o_l_a_t_i_l_e_ _a_c_i_d_i_t_y_,_c_i_t_r_i_c_ _a_c_i_d_,_r_e_s_i_d_u_a_l_ _s_u_g_a_r_,_c_h_l_o_r_i_d_e_s_,_f_r_e_e_ _s_u_l_f_u_r_ _d_i_o_x_i_d_e_,_t_o_t_a_l_ _s_u_l_f_u_r_ _d_i_o_x_i_d_e_,_d_e_n_s_i_t_y_,_p_H_,_s_u_l_p_h_a_t_e_s_,_a_l_c_o_h_o_l_,_q_u_a_l_i_t_y_
0,red,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,red,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,red,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,red,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,red,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [15]:
wine.describe()

,_f_i_x_e_d_ _a_c_i_d_i_t_y_,_v_o_l_a_t_i_l_e_ _a_c_i_d_i_t_y_,_c_i_t_r_i_c_ _a_c_i_d_,_r_e_s_i_d_u_a_l_ _s_u_g_a_r_,_c_h_l_o_r_i_d_e_s_,_f_r_e_e_ _s_u_l_f_u_r_ _d_i_o_x_i_d_e_,_t_o_t_a_l_ _s_u_l_f_u_r_ _d_i_o_x_i_d_e_,_d_e_n_s_i_t_y_,_p_H_,_s_u_l_p_h_a_t_e_s_,_a_l_c_o_h_o_l_,_q_u_a_l_i_t_y_
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


In [17]:
sorted(wine.quality.unique())

AttributeError: ignored